# Response to article

Predict: Upvotes, downvotes, number of comments, length of comments, ...  
Something interpretable would be nice.

See <a href="#summary">summary</a> at the end.

In [1]:
%matplotlib inline
import re
from time import time
from datetime import datetime
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
df_art = pd.read_csv('articles_2017_09.csv')
df_com = pd.read_csv('comments_2017_09.csv')


In [23]:
def get_dt_obj(time):
    time = time.replace('am ', '')
    # Make datetime object from string
    return datetime.strptime(time, '%d.%m.%Y %H:%M')

def time_since_epoch(time):
    return ((time)-datetime(1970,1,1)).total_seconds()

def get_hour_of_day(time):
    return (time).hour

def get_weekday(time):
    return (time).weekday()

df_com['time_dt'] = df_com['time'].apply(get_dt_obj)
df_com['time_since_epoch'] = df_com['time_dt'].apply(time_since_epoch)
df_com['hour'] = df_com['time_dt'].apply(get_hour_of_day)
df_com['weekday'] = df_com['time_dt'].apply(get_weekday) # 0 = Monday
df_com['is_answer'] = df_com['tit'].apply(lambda x: str(x).startswith('@'))
#df_com['con_len'] = df_com['con'].apply(len)
#df_com['con_num_words'] = df_com['con'].apply(lambda x: len(x.split()))
df_com['score'] = df_com['vup'] - df_com['vdo']
df_com['activity'] = df_com['vup'] + df_com['vdo']

df_com['tit'] = df_com['tit'].str.lower()
df_com['con'] = df_com['con'].str.lower()

def get_category(link):
    t = link.split('/')
    if len(t) <= 1:
        return ''
    else:
        return t[1]

df_art['cat'] = df_art['link'].apply(get_category)
df_art['cat_copy'] = df_art['cat']
df_art = pd.get_dummies(df_art, columns=['cat'])

#df_art['header'] = df_art['header'].str.lower()
#df_art['sub'] = df_art['sub'].str.lower()
#df_art['text'] = df_art['text'].str.lower()
df_art['header_len'] = df_art['header'].apply(len)
df_art['sub_len'] = df_art['sub'].apply(lambda x: len(str(x)))
df_art['text_len'] = df_art['text'].apply(lambda x: len(str(x)))
df_art['text_num_words'] = df_art['text'].apply(lambda x: len(str(x).split()))
df_art['text_n_periods'] = df_art['text'].apply(lambda x: len(str(x).split('.')))

# Left inner join
df_merge = pd.merge(left=df_com[['tId', 'score', 'activity', 'time_since_epoch', 'weekday', 'hour']], right=df_art, left_on='tId', right_on='tId')

# Remove rows with missing values
# use .count() to check for missing values
df_merge.dropna(axis=0, how='any', inplace=True)

# Get order of comments per article
df_merge = df_merge.sort_values(['tId', 'time_since_epoch']).groupby('tId')

# Get article specific statistics, group = article
def get_art_statistics(group):
    first = group.iloc[:1]['time_since_epoch']
    group['art_first_weekday'] = group.iloc[:1]['weekday']
    group['art_first_hour'] = group.iloc[:1]['hour']

    group['time_since_first'] = group['time_since_epoch'].apply(lambda x: (x - first) / 3600)

    group['art_score_sum'] = group['score'].sum()
    group['art_activity_sum'] = group['activity'].sum()
    
    return group

df_merge = df_merge.apply(get_art_statistics)
df_merge = df_merge.groupby('tId').mean().reset_index()
df_merge = df_merge.merge(df_art[['tId', 'header', 'sub', 'text']], on='tId')
df_merge.head(2)[['score', 'num_comments']]

,score,num_comments
0,110.904762,21.0
1,49.123288,73.0


In [51]:
# Prepare data

#temp2 = pd.get_dummies(df_merge[['tId', 'art_first_weekday']], prefix='art_wd_', columns=['art_first_weekday'])
# Merge here first because of index!
#df = df_merge.merge(temp2, on='tId') 
df = pd.get_dummies(df_merge, columns=['art_first_weekday'])

# Unused categories, drop them
df = df.drop(['cat_playview', 'cat_play'], axis=1)

# https://de.wikipedia.org/wiki/Liste_der_h%C3%A4ufigsten_W%C3%B6rter_der_deutschen_Sprache
stop_words = "die, der, und, in, zu, den, das, nicht, von, sie, ist, des, sich, mit, dem, dass, er, es, ein, ich, auf, so, eine, auch, als, an, nach, wie, im, für, "
stop_words += "man, aber, aus, durch, wenn, nur, war, noch, werden, bei, hat, wir, was, wird, sein, einen, welche, sind, oder, zur, um, haben, einer, mir, über, ihm, diese, einem, ihr, uns, "
stop_words += "da, zum, kann, doch, vor, dieser, mich, ihn, du, hatte, seine, mehr, am, denn, nun, unter, sehr, selbst, schon, hier, im,"
stop_words += "bis, habe, ihre, dann, ihnen, seiner, alle, wieder, meine, Zeit, gegen, vom, ganz, einzelnen, wo, muss, ohne, eines, können, sei"
stop_words = stop_words.lower().split(', ')

# add text features: text -> tfidf -> TruncatedSVD -> feed to models -> X
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

vect = TfidfVectorizer(stop_words=stop_words)
res = vect.fit_transform(df[['text']].values.ravel())

svd = TruncatedSVD(n_components=20, n_iter=5)
res = svd.fit_transform(res)

res = pd.DataFrame(res).add_prefix('svd_')
df = pd.concat([df, res], axis=1)

# Memory optimization
# Technical stuff, contributes nothing to analysis
conv = df.select_dtypes(include=['int', 'float']).apply(pd.to_numeric,downcast='unsigned')
df[conv.columns] = conv
    
# Get pearson co-efficients
df.corr()
#df.info()
#colormap = plt.cm.viridis
#plt.figure(figsize=(12,12))
#plt.title('Pearson Correlation of Features', y=1.05, size=15)
#sns.heatmap(df.corr(),linewidths=0.1,vmax=1.0, square=True, cmap=colormap, linecolor='white', annot=True)


,tId,score,activity,time_since_epoch,weekday,hour,article_id,updated,num_comments,cat_ausland,...,svd_10,svd_11,svd_12,svd_13,svd_14,svd_15,svd_16,svd_17,svd_18,svd_19
tId,1.000000,0.002489,-0.012417,-0.027862,0.007069,-0.022099,0.997714,0.010961,-0.010061,-0.012343,...,0.014859,-0.001807,-0.002397,-0.035178,-0.012574,-0.021777,-0.008962,0.004136,-0.011090,0.016443
score,0.002489,1.000000,0.792733,0.033427,-0.002571,0.065504,0.001628,-0.019505,-0.003332,0.049031,...,0.006810,0.015979,0.009467,0.099782,0.125860,-0.029136,0.035804,0.077248,-0.041240,0.050134
activity,-0.012417,0.792733,1.000000,0.045888,0.029045,0.086315,-0.013447,-0.025012,0.008002,0.103206,...,0.000202,-0.005708,0.006564,0.134554,0.102053,-0.021753,0.032799,0.093871,-0.039037,0.058750
time_since_epoch,-0.027862,0.033427,0.045888,1.000000,0.020684,0.030757,-0.027450,0.100234,0.042488,0.037357,...,0.016562,-0.019525,-0.017157,0.020851,-0.038171,0.027459,-0.002136,-0.035626,0.010537,0.044601
weekday,0.007069,-0.002571,0.029045,0.020684,1.000000,0.058002,0.009173,-0.025130,0.035074,0.003723,...,-0.004084,-0.068503,0.003651,0.025450,0.038347,-0.019010,-0.056346,-0.036331,0.025818,-0.024085
hour,-0.022099,0.065504,0.086315,0.030757,0.058002,1.000000,-0.020774,0.022068,-0.034452,-0.008135,...,0.006816,-0.078039,0.001309,-0.008851,0.009084,-0.022807,-0.071908,-0.052103,0.010690,-0.025158
article_id,0.997714,0.001628,-0.013447,-0.027450,0.009173,-0.020774,1.000000,0.014333,-0.007248,-0.013784,...,0.014876,-0.002017,-0.001201,-0.034924,-0.015438,-0.020273,-0.007437,0.001775,-0.007468,0.018904
updated,0.010961,-0.019505,-0.025012,0.100234,-0.025130,0.022068,0.014333,1.000000,-0.019521,-0.020738,...,-0.043599,0.040272,-0.032104,0.007340,-0.057555,-0.006982,-0.012921,-0.051489,0.052869,-0.017403
num_comments,-0.010061,-0.003332,0.008002,0.042488,0.035074,-0.034452,-0.007248,-0.019521,1.000000,-0.008711,...,-0.028809,0.047073,0.031504,0.149010,-0.034446,0.114192,0.067139,0.050589,-0.063363,-0.004646
cat_ausland,-0.012343,0.049031,0.103206,0.037357,0.003723,-0.008135,-0.013784,-0.020738,-0.008711,1.000000,...,-0.003368,-0.026152,-0.141561,0.022363,0.014106,0.018416,0.026813,0.004835,-0.014752,0.075248


In [57]:
# Do splits

# Drop everything related to comments
df2 = df.drop(['score', 'activity', 'vup', 'vdo', 'num_comments', #'mob',
                'art_score_sum', 'art_activity_sum', 'weekday', 'hour', #'con_len', 'con_num_words', 'is_answer',
               # remove text columns
              'text', 'header', 'sub'], axis=1)
X = df2
y = df['num_comments']
#y = df['con_len']
#y = df['vup']

from sklearn.model_selection import train_test_split, learning_curve
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
print("total data: ", len(X))

# Class to extend the Sklearn classifier
class SklearnHelper(object):
    def __init__(self, clf, seed=None, params=None):
        if seed is not None: params['random_state'] = seed
        self.clf = clf(**params)

    def train(self, x_train, y_train):
        self.clf.fit(x_train, y_train)

    def predict(self, x):
        return self.clf.predict(x)
    
    def fit(self,x,y):
        return self.clf.fit(x,y)
    
    def feature_importances(self,x,y):
        return (self.clf.fit(x,y).feature_importances_)

def get_oof(clf):
    oof_train = np.zeros((ntrain,))
    oof_test = np.zeros((ntest,))
    oof_test_skf = np.empty((NFOLDS, ntest))

    for i, (train_index, test_index) in enumerate(kf.split(X_test)):
        x_tr = X_train.values[train_index]
        y_tr = y_train.values[train_index]
        x_te = X_train.values[test_index]

        clf.train(x_tr, y_tr)

        oof_train[test_index] = clf.predict(x_te)
        oof_test_skf[i, :] = clf.predict(X_test)

    oof_test[:] = oof_test_skf.mean(axis=0)
    return oof_train.reshape(-1, 1), oof_test.reshape(-1, 1)

from sklearn.model_selection import KFold;
ntrain = X_train.shape[0]
ntest = X_test.shape[0]
SEED = 0 # for reproducibility
NFOLDS = 3 # set folds for out-of-fold prediction
kf = KFold(n_splits=NFOLDS, random_state=SEED)

#X.info()

total data:  2637


In [ ]:
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import SVR

# Put in our parameters for said classifiers
# Random Forest parameters
rf_params = {
    'n_jobs': -1,
    'n_estimators': 500,
     'warm_start': True, 
     #'max_features': 0.2,
    'max_depth': 6,
    'min_samples_leaf': 2,
    'max_features' : 'sqrt',
    'verbose': 0
}

# Extra Trees Parameters
et_params = {
    'n_jobs': -1,
    'n_estimators':500,
    #'max_features': 0.5,
    'max_depth': 8,
    'min_samples_leaf': 2,
    'verbose': 0
}

# AdaBoost parameters
ada_params = {
    'n_estimators': 500,
    'learning_rate' : 0.75
}

# Gradient Boosting parameters
gb_params = {
    'n_estimators': 500,
     #'max_features': 0.2,
    'max_depth': 5,
    'min_samples_leaf': 2,
    'verbose': 0
}

# Support Vector Classifier parameters 
svr_params = {
    'kernel' : 'linear',
    'C' : 0.025
}

# Create 5 objects that represent our 4 models
rf = SklearnHelper(clf=RandomForestRegressor, seed=SEED, params=rf_params)
et = SklearnHelper(clf=ExtraTreesRegressor, seed=SEED, params=et_params)
ada = SklearnHelper(clf=AdaBoostRegressor, seed=SEED, params=ada_params)
gb = SklearnHelper(clf=GradientBoostingRegressor, seed=SEED, params=gb_params)
svr = SklearnHelper(clf=SVR, params=svr_params) # SVR has no "random_state"

# Create our OOF train and test predictions. These base results will be used as new features
et_oof_train, et_oof_test = get_oof(et) # Extra Trees
rf_oof_train, rf_oof_test = get_oof(rf) # Random Forest
ada_oof_train, ada_oof_test = get_oof(ada) # AdaBoost 
gb_oof_train, gb_oof_test = get_oof(gb) # Gradient Boost
svr_oof_train, svr_oof_test = get_oof(svr) # Support Vector Classifier

print("Training is complete.")

In [ ]:
rf_feature = (rf.feature_importances(X_train,y_train))
et_feature = (et.feature_importances(X_train, y_train))
ada_feature = (ada.feature_importances(X_train, y_train))
gb_feature = (gb.feature_importances(X_train,y_train))

In [ ]:
cols = X_train.columns.values
# Create a dataframe with features
feature_dataframe = pd.DataFrame( {'features': cols,
     'Random Forest feature importances': rf_feature,
     'Extra Trees  feature importances': et_feature,
      'AdaBoost feature importances': ada_feature,
    'Gradient Boost feature importances': gb_feature
    })

# Create the new column containing the average of values
feature_dataframe['mean'] = feature_dataframe.mean(axis= 1) # axis = 1 computes the mean row-wise
#print(feature_dataframe.head(3))

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
y = feature_dataframe['mean'].values
x = feature_dataframe['features'].values
data = [go.Bar(
            x= x,
             y= y,
            width = 0.5,
            marker=dict(
               color = feature_dataframe['mean'].values,
            colorscale='Portland',
            showscale=True,
            reversescale = False
            ),
            opacity=0.6
        )]

layout= go.Layout(
    autosize= True,
    title= 'Barplots of Mean Feature Importance',
    hovermode= 'closest',
#     xaxis= dict(
#         title= 'Pop',
#         ticklen= 5,
#         zeroline= False,
#         gridwidth= 2,
#     ),
    yaxis=dict(
        title= 'Feature Importance',
        ticklen= 5,
        gridwidth= 2
    ),
    showlegend= False
)
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='bar-direct-labels')

In [ ]:
base_predictions_train = pd.DataFrame( {'RandomForest': rf_oof_train.ravel(),
     'ExtraTrees': et_oof_train.ravel(),
     'AdaBoost': ada_oof_train.ravel(),
      'GradientBoost': gb_oof_train.ravel()
    })
#print(base_predictions_train.head())

data = [
    go.Heatmap(
        z= base_predictions_train.astype(float).corr().values ,
        x=base_predictions_train.columns.values,
        y= base_predictions_train.columns.values,
          colorscale='Portland',
            showscale=True,
            reversescale = True
    )
]
py.iplot(data, filename='labelled-heatmap')

# Less correlation is better

In [ ]:
x_train = np.concatenate(( et_oof_train, rf_oof_train, ada_oof_train, gb_oof_train, svr_oof_train), axis=1)
x_test = np.concatenate(( et_oof_test, rf_oof_test, ada_oof_test, gb_oof_test, svr_oof_test), axis=1)

gbm = xgb.XGBClassifier(
    #learning_rate = 0.02,
 n_estimators= 100, # was 2000
 max_depth= 3, # was 4
 min_child_weight= 2,
 #gamma=1,
 gamma=0.9,                        
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread= -1,
 scale_pos_weight=1).fit(x_train, y_train)
predictions = gbm.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_error, mean_absolute_error

y_pred = predictions
print("R^2: %1.2f" % r2_score(y_test, y_pred))
print("Explained var: {:3f}".format(explained_variance_score(y_test, y_pred)))
print("Mean absolute err: {:.2f}".format(mean_absolute_error(y_test, y_pred)))
print("Mean squared err: {:.2f}".format(mean_squared_error(y_test, y_pred)))

# Residual plot
X_res = X_test['text_len']
plt.scatter(X_res, y_test, color='black', label='test data')
plt.scatter(X_res, y_pred, color='red', label='predicted')
plt.xlabel('text_len')
plt.legend(); plt.show()

### Summary
<a id="summary"></a>

In this case we did not use words at all, but text statistics. The score was a bit worse.